In [1]:
def deco(func):
  def inner():
    print('running inner()')
  return inner
@deco
def target():
  print("running target()")

target()

running inner()


In [2]:
target

<function __main__.deco.<locals>.inner()>

In [5]:
def logger(func):
  def wrapper(*args, **kwargs):
    print(f"[LOG] {func.__name__} was called with args: {args}, kwargs: {kwargs}")
    return func(*args, **kwargs)
    print(f"[LOG] {func.__name__} returned {result}")
    return result
  return wrapper
@logger
def add(a, b):
  return b + a
add(5,12)

[LOG] add was called with args: (5, 12), kwargs: {}


17

In [6]:
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')
def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()


running register(<function f1 at 0x7b0ea73b3740>)
running register(<function f2 at 0x7b0ea73b1d00>)
running main()
registry -> [<function f1 at 0x7b0ea73b3740>, <function f2 at 0x7b0ea73b1d00>]
running f1()
running f2()
running f3()


In [8]:
promos = []

def promotion(promo_func):
  """promotion{} decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity(order):
  """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
  discount = 0
  for item in order.cart:
    if item.quantity >=20:
      discount += item.total()*0.1
  return discount

@promotion
def large_order_promo(order):
  """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

def best_promo(order):
  """최대로 할인받을 금액을 반환"""
  return max(promo(order) for promo in promos)

In [9]:
def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [10]:
b=6
f1(3)

3
6


In [11]:
b = 6
def f2(a):
  print(a)
  print(b)
  b=9
f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [12]:
b=6
def f3(a):
  global b
  print(a)
  print(b)
  b=9

f3(3)

3
6


In [13]:
b

9

In [15]:
f3(3)

3
9


In [16]:
b=30
f3(3)

3
30


In [23]:
class Averager():
  def __init__(self):
    self.serise = []

  def __call__(self,new_value):
    self.serise.append(new_value)
    total = sum(self.serise)
    return total/len(self.series)

In [24]:
avg = Averager()
avg(10)

AttributeError: 'Averager' object has no attribute 'series'

In [ ]:
avg(11)

In [ ]:
avg(12)

In [25]:
def make_averager():
  serise = []
  def averager(new_value):
    serise.append(new_value)
    total = sum(serise)
    return total/len(serise)
  return averager

In [26]:
avg = make_averager()
avg(10)

10.0

In [27]:
avg(11)

10.5

In [28]:
avg(12)

11.0

In [29]:
avg.__code__.co_varnames

('new_value', 'total')

In [30]:
avg.__code__.co_freevars

('serise',)

In [31]:
avg.__closure__

(<cell at 0x7b0ea7ae8f40: list object at 0x7b0ea61ab3c0>,)

In [32]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [33]:
def make_averager():
  count = objecttotal = 0
  def averager(neww_value):
    count+=1
    total+=new_value
    return total/count
  return averager

In [34]:
avg = make_averager
avg(10)

TypeError: make_averager() takes 0 positional arguments but 1 was given

In [35]:
def make_averager():
  count = 0
  total = 0
  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count
  return averager

avg = make_averager()
print(avg(10))

10.0


In [43]:
import time
def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print(f'[%0.8fs]%s(%s)->%r'% (elapsed,name,arg_str,result))
    return result
  return clocked

In [45]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n<2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12319828s]snooze(0.123)->None
**************************************** Calling factorial(6)
[0.00000050s]factorial(1)->1
[0.00001248s]factorial(2)->2
[0.00001837s]factorial(3)->6
[0.00002531s]factorial(4)->24
[0.00003045s]factorial(5)->120
[0.00003970s]factorial(6)->720
6! = 720


In [54]:
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.perf_counter()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append('. '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s)->%r'% (elapsed,name,arg_str,result))
    return result
  return clocked

In [55]:
def make_averager():
  serise = []
  def averager(new_value):
    serise.append(new_value)
    total = sum(serise)
    return total/len(serise)
  return averager

In [56]:
@clock
def snooze(seconds):
  time.sleep(seconds)
@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*' * 40, 'Calling snooze(.123)')
  snooze(.123)
  print('*' * 40, 'Calling factorial(6)')
  print('6! =',factorial(6))

**************************************** Calling snooze(.123)
[1762132838.64440131s] snooze(0.123)->None
**************************************** Calling factorial(6)
[1762132838.52125168s] factorial(1)->1
[1762132838.52126884s] factorial(2)->2
[1762132838.52127624s] factorial(3)->6
[1762132838.52128291s] factorial(4)->24
[1762132838.52128935s] factorial(5)->120
[1762132838.52129960s] factorial(6)->720
6! = 720


In [57]:
factorial.__name__

'factorial'